# SimPy Project for IE 306.02
A call center simulation


Below we indicate package that we're gonna use.

In [1]:
import simpy
import random
import numpy as np

Define a set of globals that define the characteristics of the model instance to be simulated. This includes the seed (RANDOM_SEED) for the random number generators, and key parameters for the interarrival (i.e. mean arrival rate.)

In [2]:
class G(object): #for global variables such as number of customers
    NUMBER_OF_CUSTOMERS = 5000
    last_in_q2 = False  #boolean for indicating last customer in queue 2 (expert operator queue)
    RANDOM_SEED = 978
    INTERARRIVAL_RATE = 1/14.3 #interarrival times exp dist with mean 14.3 min 
    rnd=random.Random(RANDOM_SEED)
    lastt=0

Below code segment for the transformation to get values according to logNormal distribution.

In [3]:
m=7.2
v=2.7**2
mu=np.log(m**2/np.sqrt((v+m**2)))
sigma=np.sqrt(np.log(1+(v/m**2)))

Define the necessary set of arrays for bookkeeping

In [4]:
service1_times = [0.0]*G.NUMBER_OF_CUSTOMERS #Duration of the conversation between the customer and the operator1 (Service time)
service2_times = [0.0]*G.NUMBER_OF_CUSTOMERS #Duration of the conversation between the customer and the operator2(Service time)
queue1_w_times = [0.0]*G.NUMBER_OF_CUSTOMERS #Time spent by a customer while it waits for the operator1 (Queue waiting time Wq)
queue2_w_times = [0.0]*G.NUMBER_OF_CUSTOMERS #Time spent by a customer while it waits for the operator2 (Queue waiting time Wq)


* The class definition for the customers arriving at the modeled system. When they are created, they immediatelly initiate a call (i.e. activate the call process). 

* Once a call is initiated, this is registered as a request to the operator1 resource. The customer is put on hold until the resource activates it back. 

* When the resource is available, the customer is activated and it then initiates the ask_question process. The duration of a question-answer session is determined randomly according to a logNormal and exponential distribution.

Below code segment works as follows:
    Incoming calls are first processed by the unskilled front-desk operator who records the
    personal details of the caller and the nature of the caller's request. When the operator is
    busy the customers are put on hold (they wait in a FCFS queue). 
    Once this process is completed, the caller is directed to the expert operator, who tries to
    help the caller with his/her problem. As in the previous case, if the expert is busy the
    customers are put on hold (they wait in a FCFS queue).

In [5]:
class Customer(object): #customer class
    def __init__(self, name, env):  #constructor 
        self.env = env 
        self.name = name
        self.arrival_t = float(self.env.now) #cast to float
        self.arrival_t2 = 0  
        self.action = env.process(self.call())  #immediate call to operator1
    
    def call(self):
        #print('%s initiated a call to operator1 at %g' % (self.name, self.env.now)) 
        with operator1.request() as req:
            yield req  #request to resource operator 1
            #print('%s is assigned to operator1 at %g' % (self.name, self.env.now))
            queue1_w_times[int(self.name[5:])-1] = (float(self.env.now) - self.arrival_t) #update queue waiting times
            yield self.env.process(self.ask_question()) #request service from operator1
            #print('%s is done in  operator1 at %g' % (self.name, self.env.now))
            self.arrival_t2 = float(self.env.now) #arrival to queue of expert operator
            env.process(self.call2()) #call to expert operator
            
    
    def call2(self):
        #print('-- %s initiated a call to operator2 at %g' % (self.name, self.env.now))
        if self.name == ('Cust %s' %(G.NUMBER_OF_CUSTOMERS)): #check if customer is last or not
            G.last_in_q2 = True
        with operator2.request() as req: 
            br = G.rnd.expovariate(1/60)  #random variable with exp.dist with mean 60min
            bored = env.timeout(br) #timeout br minutes
            yield req | bored 
            if not req.triggered: #checks if customer bored and leaved the queue
                #print('++ %s got bored and left the queue at %g' % (self.name, self.env.now))
                queue2_w_times[int(self.name[5:])-1] = (float(self.env.now) - self.arrival_t2) #update queue2 waiting times
            else:
                queue2_w_times[int(self.name[5:])-1] = (float(self.env.now) - self.arrival_t2) #update queue2 waiting times
                #print('%s is assigned to an operator2 at %g' % (self.name, self.env.now))
                yield self.env.process(self.ask_question2()) #request service from expert operator
                #print('%s is done in second operator2 at %g' % (self.name, self.env.now))
                G.lastt=float(self.env.now) #update last customer service time

    def ask_question(self):#operator1 call duration
        duration = G.rnd.lognormvariate(mu, sigma) #logNormal dist. duration for operator1 service
        yield self.env.timeout(duration) #getting service
        service1_times[int(self.name[5:])-1] = float(duration) #update service time queue
    
    def ask_question2(self):#operator2 call duration
        duration2 = G.rnd.expovariate(1/10.2)#expert operator service time exp.dis with mean 10.2
        yield self.env.timeout(duration2) #getting service
        service2_times[int(self.name[5:])-1] = (duration2) #update service time queue

Above code segment>>>Service operation for operator1:
The time it takes to collect and record the details of a caller is assumed to be LogNormal
distributed with mean 7.2 and standard deviation of 2.7 minutes
Service operation for operator2:
The service time of the expert is Exponentially distributed with mean 10.2 minutes

Below code segment a class for the Expert operator to give break as explained:
The expert operator takes 3-min breaks randomly through out the day. When the
operator decides to take a break, he/she waits until completing all the customers already
waiting for her/him. If new customers arrive during operators break, they wait in the
FCFS queue until the operator serves them. The operator resumes service after the break.

In [6]:
class Expert(object): 
    def __init__(self, env): #constructor
        self.env = env
        self.action = env.process(self.giveBreak()) #call give break

    def giveBreak(self):
        while not G.last_in_q2: #checks if customer is last or not
            tired = G.rnd.expovariate(1/60) #random exp. wariable with mean 60min
            yield self.env.timeout(tired) #work "tired" minutes and deciding a break
            with operator2.request() as req:         
                #print("expert got tired at %g" % (self.env.now))
                yield req #request to get expert operator and take a break
                #print("expert took a break at %g" % (self.env.now))
                yield self.env.timeout(3) #take 3minutes break and get back to work
                #print("expert returned to work at %g" % (self.env.now))
            

Below code segment works as a customer generator as follows:
The inter-arrival times of calls are exponentially distributed with mean 14.3 min.


In [7]:
def customer_generator(env): #generating customer arrivals
    #Generate new calls that arrive at the call station.
    expert = Expert(env)#generate a expert object for 3minutes breaks.
    for i in range(G.NUMBER_OF_CUSTOMERS): #generate customers
        yield env.timeout(G.rnd.expovariate(G.INTERARRIVAL_RATE)) #inter-arrival times exp.dist with mean 14.3 min
        customer = Customer('Cust %s' %(i+1), env)  #generate customer i

In [8]:
env = simpy.Environment() #creating environment
operator1 = simpy.Resource(env, capacity = 1) #create operator1 as resource
operator2 = simpy.Resource(env, capacity = 1) #create expert operator as resorce
env.process(customer_generator(env)) #process customer generator function
env.run() #run the simulation

In [9]:
#print(queue2_w_times) 
#print(queue1_w_times) 
#print(service2_times) 

In [10]:
#Gather statistics
op1_service_t = 0 #service time for operators
op2_service_t = 0
q1_tot_w_t = 0#total waiting times for customers
q2_tot_w_t = 0
max_ratio = 0   #the maximum "total waiting time" to "total service time" ratio
for i in range(G.NUMBER_OF_CUSTOMERS):#sum and find the total values
    op1_service_t += service1_times[i]
    op2_service_t += service2_times[i]
    q1_tot_w_t += queue1_w_times[i]
    q2_tot_w_t += queue2_w_times[i]
    ratio = (queue1_w_times[i] + queue2_w_times[i]) / (service1_times[i] + service2_times[i]+queue1_w_times[i] + queue2_w_times[i]) #find ratio
    if ratio > max_ratio:  #find max ratio
        max_ratio = ratio   

#Calculate the utilization of the front-desk operator and the expert operator
op1_utilization = op1_service_t / G.lastt    #utilization of the front-desk op
op2_utilization = op2_service_t / G.lastt    #utilization of the expert op

#Calculate the average total waiting time
avg_tot_w_t = (q1_tot_w_t + q2_tot_w_t) / G.NUMBER_OF_CUSTOMERS

#Calculate the maximum "total waiting time" to "total service time" ratio
max_ratio

#Calculate the average number of people waiting to be served by the expert operator
avg_w_in_q2 = q2_tot_w_t / G.lastt

print(op1_utilization)
print(op2_utilization)
print(avg_tot_w_t)
print(max_ratio)
print(avg_w_in_q2)

0.494385955779248
0.6121770949790821
11.597779500532873
0.9618395727619385
0.5415170180098235
